# JOBSHEET 04: TEKNIK ANALISIS POSE & GEOMETRI TUBUH PADA GAMBAR 

## Install Library

In [ ]:
!pip install opencv-python mediapipe==0.10.14 numpy cvzone


## Praktikum D1 — Inisialisasi Kamera dan Akuisisi Citra

In [ ]:
import cv2, time

cap = cv2.VideoCapture(1)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka. Coba index 1/2.")

frames, t0 = 0, time.time()
while True:
    ok, frame = cap.read()
    if not ok: break
    frames += 1
    if time.time() - t0 >= 1.0:
        cv2.setWindowTitle("Preview", f"Preview (FPS ~ {frames})")
        frames, t0 = 0, time.time()
    cv2.imshow("Preview", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()


## Praktikum D2 — Deteksi Pose dan Analisis Sudut Tubuh 

In [ ]:
import cv2, numpy as np
from cvzone.PoseModule import PoseDetector

cap = cv2.VideoCapture(1)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka.")

detector = PoseDetector(staticMode=False, modelComplexity=1,
                        enableSegmentation=False, detectionCon=0.5,
                        trackCon=0.5)

while True:
    success, img = cap.read()
    img = detector.findPose(img)
    lmList, bboxInfo = detector.findPosition(img, draw=True, bboxWithHands=False)
    
    if lmList:
        center = bboxInfo["center"]
        cv2.circle(img, center, 5, (255, 0, 255), cv2.FILLED)

        # Hitung sudut siku kiri
        angle, img = detector.findAngle(lmList[11][0:2],
                                        lmList[13][0:2],
                                        lmList[15][0:2],
                                        img=img,
                                        color=(0, 0, 255),
                                        scale=10)
        cv2.putText(img, f"Angle: {angle:.1f}", (50,50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    cv2.imshow("Pose + Angle", img)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()


## Praktikum D3 — Deteksi Wajah dan Analisis Kedipan Mata 

In [ ]:
import cv2, numpy as np
from cvzone.FaceMeshModule import FaceMeshDetector

L_TOP, L_BOTTOM, L_LEFT, L_RIGHT = 159, 145, 33, 133

def dist(p1, p2): return np.linalg.norm(np.array(p1) - np.array(p2))

cap = cv2.VideoCapture(1)
detector = FaceMeshDetector(maxFaces=1)
blink_count, closed_frames = 0, 0
CLOSED_FRAMES_THRESHOLD = 3
EYE_AR_THRESHOLD = 0.20
is_closed = False

while True:
    ok, img = cap.read()
    if not ok: break
    img, faces = detector.findFaceMesh(img, draw=True)
    if faces:
        face = faces[0]
        v = dist(face[L_TOP], face[L_BOTTOM])
        h = dist(face[L_LEFT], face[L_RIGHT])
        ear = v / (h + 1e-8)
        cv2.putText(img, f"EAR: {ear:.2f}", (20,40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2)

        if ear < EYE_AR_THRESHOLD:
            closed_frames += 1
            if closed_frames >= CLOSED_FRAMES_THRESHOLD and not is_closed:
                blink_count += 1
                is_closed = True
        else:
            closed_frames, is_closed = 0, False

        cv2.putText(img, f"Blink: {blink_count}", (20,70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

    cv2.imshow("FaceMesh + EAR", img)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()


## Praktikum D4 — Deteksi Tangan dan Penghitungan Jumlah Jari

In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector

cap = cv2.VideoCapture(1)
detector = HandDetector(maxHands=1)

while True:
    ok, img = cap.read()
    if not ok: break
    hands, img = detector.findHands(img, draw=True)
    if hands:
        fingers = detector.fingersUp(hands[0])
        count = sum(fingers)
        cv2.putText(img, f"Fingers: {count}", (20,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    cv2.imshow("Hands", img)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()


## Praktikum D5 — Pengenalan Gestur Tangan (Hand Gesture Recognition)

In [ ]:
import cv2, numpy as np
from cvzone.HandTrackingModule import HandDetector

def dist(a,b): return np.linalg.norm(np.array(a)-np.array(b))

def classify_gesture(hand):
    lm = hand["lmList"]
    wrist, thumb_tip, index_tip, middle_tip, ring_tip, pinky_tip = \
        np.array(lm[0][:2]), np.array(lm[4][:2]), np.array(lm[8][:2]), \
        np.array(lm[12][:2]), np.array(lm[16][:2]), np.array(lm[20][:2])
    if dist(thumb_tip, index_tip) < 35: return "OK"
    if (thumb_tip[1] < wrist[1]-40): return "THUMBS UP"
    return "UNKNOWN"

cap = cv2.VideoCapture(1)
detector = HandDetector(maxHands=1)

while True:
    ok, img = cap.read()
    hands, img = detector.findHands(img, draw=True)
    if hands:
        label = classify_gesture(hands[0])
        cv2.putText(img, f"Gesture: {label}", (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)

    cv2.imshow("Gestures", img)
    if cv2.waitKey(1) & 0xFF == ord('q'): break

cap.release()
cv2.destroyAllWindows()


## Praktikum D6 — Analisis Gerakan Tubuh dan Penghitung Aktivitas

In [ ]:
import cv2, numpy as np
from collections import deque
from cvzone.PoseModule import PoseDetector

MODE = "squat"
cap = cv2.VideoCapture(1)
detector = PoseDetector()
count, state = 0, "up"
debounce = deque(maxlen=6)

def ratio_pushup(lm):
    sh = np.array(lm[11][1:3])
    wr = np.array(lm[15][1:3])
    hp = np.array(lm[23][1:3])
    return np.linalg.norm(sh - wr) / (np.linalg.norm(sh - hp) + 1e-8)

while True:
    ok, img = cap.read()
    if not ok: break
    img = detector.findPose(img)
    lmList, _ = detector.findPosition(img, draw=False)
    flag = None
    if lmList:
        if MODE == "squat":
            angL, img = detector.findAngle(lmList[23][0:2], lmList[25][0:2], lmList[27][0:2], img=img)
            angR, img = detector.findAngle(lmList[24][0:2], lmList[26][0:2], lmList[28][0:2], img=img)
            ang = (angL + angR) / 2
            if ang < 80: flag = "down"
            elif ang > 160: flag = "up"
        else:
            r = ratio_pushup(lmList)
            if r < 0.85: flag = "down"
            elif r > 1.0: flag = "up"
        debounce.append(flag)
        if debounce.count("down") >= 4 and state == "up":
            state = "down"
        if debounce.count("up") >= 4 and state == "down":
            state, count = "up", count + 1

    cv2.putText(img, f"Mode: {MODE} | Count: {count}", (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    cv2.imshow("Activity Counter", img)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): break
    if key == ord('m'): MODE = "pushup" if MODE == "squat" else "squat"

cap.release()
cv2.destroyAllWindows()
